In [1]:
import os, pathlib

file_path = os.path.dirname(os.path.realpath("__file__"))
home_dir = pathlib.Path(file_path).parent.parent
os.chdir(home_dir)

In [2]:
import pandas as pd
from src.utils import SAVED_EVAL_BASE_PATH

df = pd.read_csv(SAVED_EVAL_BASE_PATH.joinpath("evaluation_result.csv"))
df

,Unnamed: 0,dataset,layer,eval,classifier,refAE,0,1,5,10,25,50,99,99.9,99.99,100
0,0,fashion_mnist,my_dense,total_loss,0.228498,7.124279,21.836708,21.761026,20.813839,19.769808,16.493725,11.121116,0.631990,0.431054,0.365682,0.347074
1,1,fashion_mnist,my_dense,reconstruction_loss,NaN,7.124279,21.836708,21.972736,21.874737,21.904966,21.857809,21.926941,30.009632,90.457542,275.725006,375.310516
2,2,fashion_mnist,my_dense,classification_loss,0.228498,NaN,0.875324,0.802315,0.656808,0.553377,0.401475,0.315300,0.335246,0.340938,0.338143,0.347074
3,3,fashion_mnist,my_dense,evaluation_accuracy,0.920600,0.888700,0.903300,0.904600,0.905400,0.905600,0.907600,0.907800,0.913900,0.914000,0.912600,0.912900
4,4,fashion_mnist,my_conv,total_loss,0.228498,1.125318,3.543053,3.508368,3.383557,3.239841,2.740736,1.926461,0.335954,0.295887,0.368075,0.332982
5,5,fashion_mnist,my_conv,reconstruction_loss,NaN,1.125318,3.543053,3.540704,3.545505,3.566517,3.560159,3.585461,8.962866,26.455465,100.750717,93.023277
6,6,fashion_mnist,my_conv,classification_loss,0.228498,NaN,0.308961,0.307043,0.306536,0.299761,0.282462,0.267461,0.248813,0.269701,0.358036,0.332982
7,7,fashion_mnist,my_conv,evaluation_accuracy,0.920600,0.908000,0.904500,0.892100,0.904600,0.905800,0.908000,0.908400,0.908400,0.909900,0.910800,0.906400
8,8,mnist,my_dense,total_loss,0.037522,3.131160,20.522276,20.367018,19.583939,18.485197,15.485126,10.302245,0.294454,0.171916,0.148933,0.129434
9,9,mnist,my_dense,reconstruction_loss,NaN,3.131160,20.522276,20.571531,20.608286,20.526329,20.612139,20.515209,24.082470,87.526779,304.325165,312.463287
